In [ ]:
%matplotlib widget

import numpy as np
from lotr import DATASET_LOCATION
from lotr import plotting as pltltr
from matplotlib import pyplot as plt
from tifffile import imread

COLS = pltltr.COLS

In [ ]:
DATASET_LOCATION

In [ ]:
isynapses_file = DATASET_LOCATION.parent / "anatomy" / "reconstruction" / "synapses.xml"

import xml.etree.ElementTree as et

from lotr.em.core import EmNeuron
from lotr.em.loading import load_skeletons_from_zip

path = "/Users/luigipetrucco/Desktop/cell1_synapse_locations/annotation.xml"
xtree = et.parse(path)
node = xtree.getroot()
synapses = []
for n in list(node):  # ugly loop
    if n.tag == "thing":
        synapses.append(EmNeuron(n))

# Load the EM and mirror neurons on one side:
em_path = DATASET_LOCATION.parent / "anatomy" / "all_ahb.k.zip"

neurons_list = load_skeletons_from_zip(em_path)

for n in neurons_list:
    n.mirror_right = True

# Select neurons with projections internal to the dIPN:
sel_neurons = [
    n
    for n in neurons_list
    if ("bilat" in n.comments or "bilatdend" in n.comments)
    and not "proj" in n.comments
    and "dipn" in n.comments.split("_")
]

neurons_list_not_mirr = load_skeletons_from_zip(em_path)

# Select neurons with projections internal to the dIPN:
sel_neurons_not_mirr = [
    n
    for n in neurons_list_not_mirr
    if ("bilat" in n.comments or "bilatdend" in n.comments)
    and not "proj" in n.comments
    and "dipn" in n.comments.split("_")
]

In [ ]:
from bg_atlasapi.core import Atlas

atlas = Atlas(DATASET_LOCATION.parent / "anatomy" / "ipn_zfish_0.5um_v1.8")

#  Specify axes limits over all dimensions:
bs = dict(frontal=(30, 180), vertical=(-4, 170), sagittal=(-50, 120))

plotter = pltltr.AtlasPlotter(
    atlas=atlas,
    structures=["ipn", "dipn"],
    mask_slices=dict(frontal=slice(0, 120)),
    bounds_dict=dict(
        frontal=[bs["vertical"], bs["frontal"]],
        horizontal=[bs["sagittal"], bs["frontal"]],
        sagittal=[bs["vertical"], bs["sagittal"]],
    ),
)

In [ ]:
sel_neuron_id = "c008"

neuron = [n for n in sel_neurons if n.id == sel_neuron_id][0]

lw = 0.8
f, axs = plotter.generate_projection_plots()
main_c = ".5"  # COLS["qualitative"][i % len(COLS["qualitative"])]
plotter.plot_neurons(
    axs,
    neuron,
    select="dendrites",
    c=pltltr.shift_lum(main_c, -0.05),
    lw=lw,
    soma_s=30,
    label="__nolegend__",
)
plotter.plot_neurons(
    axs,
    neuron,
    select="axon",
    c=pltltr.shift_lum(main_c, 0.2),
    lw=lw,
    soma_s=0,
    rasterized=True,
    label="__nolegend__",
)

for syn, lab, c in zip(
    synapses, ["presyn. terminals", "postsyn. terminals"], ["C2", "C6"]
):
    plotter.axs_scatterplot(
        axs, syn.coords_ipn, color=c, s=1, label="__nolabel__", zorder=100
    )
    plotter.axs_scatterplot(
        axs, np.ones((1, 3)) * 1000, color=c, s=20, label=lab, zorder=100
    )

axs[1].set_title(neuron.id)
axs[1].legend(handlelength=0.0)

In [ ]:
pltltr.savefig("synapses_plot", folder="S11b")